In [49]:
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
import pandas as pd
import re
import numpy as np
import tabula
import os

In [73]:
config_dir = "/Users/imobassey/Desktop/DevOps/Git/multinational-retail-data-centralisation299/"  
db_connector = DatabaseConnector(config_dir)
data_extract = DataExtractor()
data_cleaner = DataCleaning()
api_keys = db_connector.read_api_keys()
number_of_stores = data_extract.list_number_of_stores(api_keys['NUMBER_OF_STORES_ENDPOINT'], api_keys['headers'])
print(f" The number of stores are: ", number_of_stores)



 The number of stores are:  451


In [74]:
stores_df = data_extract.retrieve_stores_data(api_keys['STORE_DETAILS_ENDPOINT'], api_keys['headers'], number_of_stores)
stores_df

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe


In [77]:
address = list(stores_df['address'].unique())
address

['N/A',
 'Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe',
 'Heckerstraße 4/5\n50491 Säckingen, Landshut',
 '5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury',
 'Studio 6\nStephen landing\nSouth Simon\nB77 2WA, Belper',
 'Flat 92u\nChristian harbors\nPort Charlotte\nN57 8FJ, Gainsborough',
 '7 Gillian rue\nWest Robertside\nPH4 8NY, Rutherglen',
 'Lilija-Heß-Allee 660\n34566 Regensburg, Stuttgart',
 '510 Jill Mill\nSouth Laura, FL 38723, Kaukauna',
 '3 Lee valleys\nWest Janetview\nDY4M 2RL, Hartley',
 '8 Gareth skyway\nSimmonsview\nSW4 7PL, Rutherglen',
 'Bergerstr. 1/3\n59183 Seelow, Siegburg',
 'Flat 37\nBennett expressway\nNew Charlotte\nSY8R 5WE, Devizes',
 '520 Fisher Inlet Suite 594\nPort Eric, IL 71753, Crystal Lake',
 'Herma-Rädel-Gasse 29\n74557 Fulda, Halstenbek',
 '4 Teresa lodge\nWest Mathewchester\nSP39 3UZ, Lancing',
 '74 Mills route\nLaurahaven\nPL4R 6HA, Newbury',
 '9 Ryan springs\nLake Allan\nE0 1AW, Lymington',
 'Studio 80d\nHolly cliffs\nIanport\nG4F 4HR,

In [81]:
#  Correct the 'continent' column
stores_df['continent'] = stores_df['continent'].replace({'eeEurope': 'Europe'})
valid_continents = ['Europe', 'America']
stores_df['continent'] = stores_df['continent'].apply(lambda x: x if x in valid_continents else np.nan)

# Removing rows with non-numeric longitude and latitude
stores_df['longitude'] = pd.to_numeric(stores_df['longitude'], errors='coerce')
stores_df['latitude'] = pd.to_numeric(stores_df['latitude'], errors='coerce')

#  Handle missing values (fill None with np.nan)
stores_df = stores_df.replace('N/A', np.nan).replace('', np.nan)
stores_df = stores_df.applymap(lambda x: np.nan if x in ['K0ODETRLS3', 'K8CXLZDP07', 'UXMWDMX1LC', '3VHFDNP8ET', '9D4LK7X4LZ', 'D23PCWSM6S', '36IIMAQD58', 'NN04B3F6UQ', 'JZP8MIJTPZ', 'B3EH2ZGQAV', '1WZB1TE1HL'] else x)

#  Ensure correct data types
stores_df['staff_numbers'] = pd.to_numeric(stores_df['staff_numbers'], errors='coerce')
stores_df['opening_date'] = pd.to_datetime(stores_df['opening_date'], errors='coerce')

#  Drop rows with all NaNs in critical columns
stores_df = stores_df.dropna(subset=['address', 'longitude', 'latitude', 'locality', 'store_code', 'staff_numbers', 'opening_date', 'store_type', 'country_code', 'continent'])


/var/folders/yq/r9wcdltx4655gh042jpyr47c0000gn/T/ipykernel_39013/3746575989.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stores_df = stores_df.applymap(lambda x: np.nan if x in ['K0ODETRLS3', 'K8CXLZDP07', 'UXMWDMX1LC', '3VHFDNP8ET', '9D4LK7X4LZ', 'D23PCWSM6S', '36IIMAQD58', 'NN04B3F6UQ', 'JZP8MIJTPZ', 'B3EH2ZGQAV', '1WZB1TE1HL'] else x)


In [83]:
stores_df

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34.0,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92.0,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26000,None,Westbury,WE-1DE82CEE,69.0,2014-01-02,Super Store,-2.18750,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.02330,None,Belper,BE-18074576,35.0,2019-09-09,Local,-1.48119,GB,Europe
5,5,Flat 92u\nChristian harbors\nPort Charlotte\nN...,53.38333,None,Gainsborough,GA-CAD01AC2,36.0,1995-05-15,Local,-0.76667,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
445,445,"Flat 7\nStephanie lake\nMorrisside\nHP8 8LH, C...",50.76306,None,Cowes,CO-473A9FBB,94.0,2008-06-08,Super Store,-1.29772,GB,Europe
446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.20000,None,Kirchlengern,KI-78096E8C,61.0,2005-05-12,Super Store,8.63333,DE,Europe
448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33.0,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35.0,2020-10-17,Local,8.70745,DE,Europe
